# Import Libs

In [1]:
import yfinance as yf
import pandas as pd
from sklearn.ensemble import IsolationForest
import smtplib
from email.mime.text import MIMEText
import datetime


# Step 1: Fetch Data

In [60]:
def fetch_financial_data(ticker, start_date, end_date):
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date)
        data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
        data.dropna(inplace=True)
        return data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

# Get historical (Day 0) and new (Day 1) data
ticker = "AAPL"
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=30)
start_date = today - datetime.timedelta(days=1000)  # 30 days of historical data

print("Fetching Day 0 (historical) data...")
day0_data = fetch_financial_data(ticker, start_date, yesterday)

print("Fetching Day 1 (latest) data...")
day1_data = fetch_financial_data(ticker, yesterday, today)

if day0_data is None or day1_data is None:
    print("Data fetch failed. Exiting.")
    exit()

Fetching Day 0 (historical) data...
Fetching Day 1 (latest) data...


In [61]:
day0_data

,Open,High,Low,Close,Volume
Date,,,,,
2022-06-01 00:00:00-04:00,147.726551,149.539884,145.538738,146.553818,74286600
2022-06-02 00:00:00-04:00,145.686566,149.076691,144.730629,149.017563,72348100
2022-06-03 00:00:00-04:00,144.770030,145.824522,142.365421,143.272079,88570300
2022-06-06 00:00:00-04:00,144.898167,146.415847,142.799046,144.021072,71598400
2022-06-07 00:00:00-04:00,142.257034,146.839606,142.010659,146.553818,67808200
...,...,...,...,...,...
2025-01-17 00:00:00-05:00,231.865066,232.034878,228.229064,229.727417,68488300
2025-01-21 00:00:00-05:00,223.753984,224.173521,219.139063,222.395477,98070400
2025-01-22 00:00:00-05:00,219.548596,223.873842,219.548596,223.584167,64126500


In [62]:
day1_data

,Open,High,Low,Close,Volume
Date,,,,,
2025-01-27 00:00:00-05:00,223.773962,231.895022,223.733997,229.607544,94863400
2025-01-28 00:00:00-05:00,230.596471,239.926210,230.556507,237.998322,75707600
2025-01-29 00:00:00-05:00,233.862872,239.596573,233.752992,239.097122,45486100
2025-01-30 00:00:00-05:00,238.407871,240.525538,236.949483,237.329056,55658300
2025-01-31 00:00:00-05:00,246.918527,246.918527,233.183628,235.740814,101075100
2025-02-03 00:00:00-05:00,229.737419,231.575395,225.452122,227.759583,73063300
2025-02-04 00:00:00-05:00,227.000419,232.873966,226.401072,232.544327,45067300
2025-02-05 00:00:00-05:00,228.279016,232.414468,228.019307,232.214691,39620300
2025-02-06 00:00:00-05:00,231.035979,233.543232,230.176923,232.963867,29925300


# Step 2: Feature Engineering

In [63]:
def preprocess_data(data):
    data["5_day_ma"] = data["Close"].rolling(window=5).mean()
    data["z_score"] = (data["Close"] - data["Close"].mean()) / data["Close"].std()
    data.dropna(inplace=True)  # Remove NaNs after rolling mean
    return data

day1_data = preprocess_data(day1_data)
day0_data = preprocess_data(day0_data)

# Step 3: Train Anomaly Detection Model

In [64]:
print("Training Isolation Forest model...")

features = ["Close", "Volume", "5_day_ma"]
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(day0_data[features])

Training Isolation Forest model...


IsolationForest(contamination=0.05, random_state=42)

# Step 4: Detect Anomalies in New Data

In [68]:
# Ensure Day 1 data is not empty
if day1_data.empty:
    print("⚠️ No new data available for Day 1. Skipping anomaly detection.")
    exit()

# Ensure enough samples exist after preprocessing
if day1_data.shape[0] < 1:
    print("⚠️ No valid samples after feature engineering. Skipping anomaly detection.")
    exit()

# Detect anomalies
print("Detecting anomalies in Day 1 data...")
day1_data["anomaly"] = model.predict(day1_data[features])

# Filter anomalies
anomalies = day1_data[day1_data["anomaly"] == -1]

if anomalies.empty:
    print("✅ No anomalies detected today.")
else:
    print(f"🚨 {len(anomalies)} anomalies detected! Sending alerts...")


Detecting anomalies in Day 1 data...
🚨 1 anomalies detected! Sending alerts...


# Step 5: Send Email Alerts

In [ ]:
if anomalies.empty:
    print("No anomalies detected today.")
else:
    print(f"🚨 {len(anomalies)} anomalies detected! Sending alerts...")

def send_email_alert(body, subject="Financial Data Anomaly Detected!", to_email="shripal12m@gmail.com"):
    sender_email = "test@gmail.com"
    sender_password = "test_password"  # Generate App Password for Gmail

    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = to_email

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, to_email, msg.as_string())
        print("✅ Alert email sent successfully!")
    except Exception as e:
        print(f"⚠️ Error sending email: {e}")

    # Email message
    subject = "🚨 Financial Anomaly Detected!"
    body = f"Anomalies detected in {ticker}:\n\n{anomalies.to_string()}"
    recipient_email = "recipient@example.com"  # Change to recipient's email

    send_email_alert(subject, body, recipient_email)
    print("Alert email sent successfully!")


Detecting anomalies in Day 1 data...
🚨 1 anomalies detected! Sending alerts...


# Step 6: Save Results

In [ ]:
anomalies.to_csv("./output/anomalies_detected.csv", index=True)
print("✅ Anomalies saved to anomalies_detected.csv")


✅ Anomalies saved to anomalies_detected.csv


In [ ]:
msft = yf.Ticker("MSFT")
barc = yf.Ticker("BARC.L")
aapl = yf.Ticker("AAPL")
goog = yf.Ticker("GOOG")
amzn = yf.Ticker("AMZN")
tsla = yf.Ticker("TSLA")

In [19]:
barc.info 

{'address1': '1 Churchill Place',
 'city': 'London',
 'zip': 'E14 5HP',
 'country': 'United Kingdom',
 'phone': '44 20 7116 1000',
 'website': 'https://home.barclays',
 'industry': 'Banks - Diversified',
 'industryKey': 'banks-diversified',
 'industryDisp': 'Banks - Diversified',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': 'Barclays PLC provides various financial services in the United Kingdom, Europe, the Americas, Africa, the Middle East, and Asia. The company operates through Barclays UK; Barclays UK Corporate Bank; Barclays Private Bank and Wealth Management; Barclays Investment Bank; and Barclays US Consumer Bank segments. It offers financial services, such as retail banking, credit cards, wholesale banking, investment banking, wealth management, and investment management services, as well as lending products. In addition, the company engages in securities dealing activities and issuing of credit

In [4]:
msft.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [20]:
aapl.info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [24]:
aapl.analyst_price_targets

{'current': 247.1,
 'high': 325.0,
 'low': 197.0,
 'mean': 252.22575,
 'median': 257.5}